==============================================================================
==============================================================================

<h1>Build a small Retrieval Augmented Generation application running on PDF files</h1>

## **Services used:**

- **LLM:** Oracle GenAI - Cohere Command model
- **Integration:** LangChain
- **Vector database:** FAISS (in-memory vector database)
- **UI:** Gradio

==============================================================================
==============================================================================

> ## **Please follow the below steps carefully**

==============================================================================
==============================================================================


## **Step 1 - Install a Conda environment**

Please follow the below steps in order.

1. In the top left corner, click on **"Launcher"**. When you don't see the "Launcher" tab, click on the blue + button.
2. Scroll down a little and click on **"Terminal"**. This will open a new terminal.
3. **Copy the below snippet**, paste in the terminal, and hit "Enter" on your keyboard. This will install the TensorFlow 2.8 for GPU on Python 3.8 Conda environment.

In [ ]:
## Snippet:
odsc conda install -s tensorflow28_p38_gpu_v1

- 4. Please wait untill the terminal has finished installing the Conda environment. **This may take 2 - 4 minutes**

==============================================================================
==============================================================================

## **Step 2 - Install additional Python packages in the Conda environment**

Please follow the below steps in order.

- 1. Make sure the terminal has finished installing the conda environment! 
- 2. In the top right corner, open the kernel by clicking on **"Python 3 (ipykernel)"**. This opens a list, select the conda environment you just installed, which is **"Python[conda env:tensorflow28_p38_gpu_v1"**

When the tensorflow28_p38_gpu_v1 conda is not visible yet, please refresh the page or check the terminal that the conda has properly installed.

- 3. When you select the tensorflow28_p38_gpu_v1 in the top right kernel, run the below cell. This will install additional Python packages in the conda environment.

In [ ]:
#Run this cell when you have selected the tensorflow28_p38_gpu_v1 in the top right kernel
!pip install accelerate tiktoken openai torch accelerate safetensors sentence-transformers faiss-gpu bitsandbytes pypdf typing-extensions PyPDF2 
!pip install tokenizers --upgrade
!pip install transformers -U
!pip install langchain -U
!pip install langchain-community --upgrade
!pip install gradio --upgrade
!pip install sqlalchemy --upgrade

==============================================================================
==============================================================================

## **Step 3 - Run the below cells and click on the public URL button to open the application**

- 1. Run the below cell, this will display an overview of the steps the application takes.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
image = mpimg.imread("overview.png")
plt.figure(figsize=(20,6))
plt.imshow(image)
plt.show()

- 2. Run the below cell. Scroll down to the end of the cell and click on the URL after **Running on public URL**
- 3. To cancel the application, click on the square button (interrupt Kernel)

In [ ]:
import uuid
import langchain_community
import langchain
import oci
import gradio as gr
import torch
import PyPDF2 # pdf reader
import time
import oci
import ads
import os
from pypdf import PdfReader
from io import BytesIO
from langchain.prompts import PromptTemplate 
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import FAISS 
from langchain.chains import RetrievalQA 
from langchain.memory import ConversationBufferMemory 
from langchain.document_loaders import PyPDFDirectoryLoader 
from transformers import AutoTokenizer
from langchain.chains import ConversationChain
from langchain_community.llms import OCIGenAI
import transformers
import tokenizers
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

## set the compartment id
compartment_ocid="ocid1.compartment.oc1..aaaaaaaafb4m3phzplh5pxmkwpwmgyavgrjz2ta5xmegdzd26nk3xjmenadq"                  ## Do not change or add your own compartment OCID

##########################################################################################################################################
########################################################################################################################################## Settings
##########################################################################################################################################

max_return_from_vector = 4
CHUNK_SIZE = 1000

##########################################################################################################################################
########################################################################################################################################## Embeddings Model
##########################################################################################################################################


# Using HuggingFaceEmbeddings with the chosen embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",model_kwargs = {"device": "cpu"})                                                                         

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

##########################################################################################################################################
########################################################################################################################################## Load Oracle GenAI service (Cohere LLM) 
##########################################################################################################################################

def load_llm():
    
    print("Start load GenAI")
    
    llm = OCIGenAI(
    model_id="cohere.command",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id=compartment_ocid,
    auth_type="RESOURCE_PRINCIPAL",          #note. We are using resource principal for authentication. Dynamic group and policy should be in order
    model_kwargs={"temperature": 0.7, "top_p": 0.75, "max_tokens": 200},
    )

    return llm

##########################################################################################################################################
########################################################################################################################################## Create history
##########################################################################################################################################

def add_text(history, text):

    print("Start add text")
    if not text:
        raise gr.Error('Enter text')
    history = history + [(text, '')]
    return history

##########################################################################################################################################
########################################################################################################################################## Upload files
##########################################################################################################################################

def upload_file(files):
    print(type(files))
    print("done with upload_files")
    
    files = files[0].name
    print(files)

    return files

##########################################################################################################################################
########################################################################################################################################## Process files
##########################################################################################################################################

def process_file(files):

    print("start process_files")
    """Function reads each loaded file, and extracts text from each of their pages
    The extracted text is store in the 'text variable which is the passed to the splitter
    to make smaller chunks necessary for easier information retrieval and adhere to max-tokens(4096)"""

    
    ### Read PDF pages and extract text
    pdf_text = ""
    for file in files:
        pdf = PyPDF2.PdfReader(file.name)
        print(pdf)
        for page in pdf.pages:
            pdf_text += page.extract_text()


    # split PDF files/text smaller chunks
    print("Start chuncking")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=200)
    splits = text_splitter.create_documents([pdf_text])

    # create a FAISS vector store db. Create embeddings and adding to faiss
    print("Store embeddings in FAISS")
    vectorstore_db = FAISS.from_documents(splits, embeddings)
    

    #create a custom prompt
    custom_prompt_template = """You have been given the following documents to answer the user's question.
    If you do not have information from the files given to answer the questions just say I don't have information from the given files to answer. Do not try to make up an answer. Only use maximum 2 sentences to answer the question.
    Context: {context}
    History: {history}
    Question: {question}

    Helpful answer:
    """
    
    ## Full prompt template, including history, context
    prompt = PromptTemplate(template=custom_prompt_template, input_variables=["question", "context", "history"])

    # set QA chain with memory
    qa_chain_with_memory = RetrievalQA.from_chain_type(llm=load_llm(),
                                                       chain_type='stuff',
                                                       return_source_documents=True,
                                                       retriever=vectorstore_db.as_retriever(search_kwargs={"k": max_return_from_vector}),
                                                                           chain_type_kwargs={"verbose": False,
                                                                                              "prompt": prompt,
                                                                                              "memory": ConversationBufferMemory(
                                                                                                          input_key="question",
                                                                                                          memory_key="history",
                                                                                                          return_messages=True) })
   
    print("returning qa_chain_with_memory")
    
    return qa_chain_with_memory

##########################################################################################################################################
########################################################################################################################################## Main
##########################################################################################################################################

def generate_bot_response(history,query, btn):
    
    print("Start generate_bot_response")
    
    #query append
    query = query + ", please use only 1 sentence in your answer"
    
    qa_chain_with_memory = process_file(btn) # run the qa chain with files from upload
    bot_response = qa_chain_with_memory({"query": query})
    
    print("--" *50)
    print("Bot response is")
    print(bot_response)
    print("--" *50)
 
    for char in bot_response['result']:
        history[-1][-1] += char
        time.sleep(0.05)
        yield history,''


##########################################################################################################################################
########################################################################################################################################## Gradio
##########################################################################################################################################

with gr.Blocks() as demo:
    css=".contain { display: flex !important; flex-direction: column !important; }"
    "#component-0, #component-3, #component-10, #component-8  { height: 100% !important; }"
    "#chatbot { flex-grow: 1 !important; overflow: auto !important;}"
    "#col { height: 100vh !important; }"
    
        
    
    with gr.Row():
            with gr.Row():
              # Chatbot interface
              chatbot = gr.Chatbot(label="Oracle GenAI",
                                   value=[],
                                   elem_id='chatbot',
                                   render=True,
                                    bubble_full_width=False)
                
                
            with gr.Column():
                # PDF upload button
                btn = gr.UploadButton("📁 Upload a PDF",
                                      file_types=[".pdf"],
                                      file_count="multiple")
                with gr.Row():
                  # Uploaded PDFs window
                  files = gr.File(label="Your PDFs")

            

    with gr.Column():
        with gr.Column():
          # Ask question input field
          txt = gr.Text(show_label=False, placeholder="Enter question")

        with gr.Column():
          # button to submit question to the bot
          submit_btn = gr.Button('Ask')
    

    # Event handler for uploading a PDF
    btn.upload(fn=upload_file, inputs=[btn], outputs=[files])

    # Event handler for submitting text question and generating response
    submit_btn.click(
        fn= add_text,
        inputs=[chatbot, txt],
        outputs=[chatbot],
        queue=True
        ).success(
          fn=generate_bot_response,
          inputs=[chatbot, txt, btn],
          outputs=[chatbot, txt]
        ).success(
          fn=upload_file,
          inputs=[btn],
          outputs=[files]
        )

if __name__ == "__main__":
    demo.queue()
    demo.launch(share=True, debug=True) # launch app

## **Example conversation**

Why should I travel to Groningen?

Yes please. What are new innovations in Groningen?

Yes. What does advancing hydrogen vehicles mean?

No, thank you!